<a href="https://colab.research.google.com/github/YorkJong/vistock/blob/feature%2Fibd/notebooks/ibd_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install vistock from GitHub (免費版Colab會固定時間清掉安裝的東西，所以重安裝是新連線後最先要做的事)

In [2]:
%pip install "git+https://github.com/YorkJong/vistock.git@feature/ibd"

  Cloning https://github.com/YorkJong/vistock.git (to revision feature/ibd) to /tmp/pip-req-build-o3mz1rxn
  Running command git clone --filter=blob:none --quiet https://github.com/YorkJong/vistock.git /tmp/pip-req-build-o3mz1rxn
  Running command git checkout -b feature/ibd --track origin/feature/ibd
  Switched to a new branch 'feature/ibd'
  Branch 'feature/ibd' set up to track remote branch 'feature/ibd' from 'origin'.
  Resolved https://github.com/YorkJong/vistock.git to commit a30c33c01ed74b68f9614149929d33ea3a01b8ba
  Preparing metadata (setup.py) ... done


### IBD RS Rating

In [6]:
import os
from vistock import ibd
from vistock.stock_indices import get_sox_tickers


def main(min_percentile=80, out_dir='out'):
    '''
    Args:
        min_percentile (int, optional): The minimum percentile for a stock to be
            included in the rankings. Defaults to 80.
        out_dir (str, optional): The output directory to store CSV tables.
            Defaults to 'out'
    '''
    rank_stock, rank_indust = ibd.rankings(get_sox_tickers())

    if rank_stock.empty or rank_indust.empty:
        print("Not enough data to generate rankings.")
        return

    print('Stock Rankings:')
    print(rank_stock[rank_stock[ibd.TITLE_PERCENTILE] >= min_percentile])

    print('\n\nIndustry Rankings:')
    print(rank_indust)

    if not os.path.exists(out_dir):
        os.makedirs(out_dir)

    # Save to CSV
    print("\n***")
    for table, kind in zip([rank_stock, rank_indust],
                           ['stocks', 'industries']):
        filename = f'sox_{kind}.csv'
        table.to_csv(os.path.join(out_dir, filename), index=False)
        print(f"Your '{filename}' is in the output folder.")
    print("***\n")


if __name__ == "__main__":
    main()

Stock Rankings:
   Ticker      Sector                             Industry  Relative Strength  \
19   NVDA  Technology                       Semiconductors             155.44   
26    TSM  Technology                       Semiconductors             125.89   
5    AVGO  Technology                       Semiconductors             122.61   
17   MPWR  Technology                       Semiconductors             119.84   
6    COHR  Technology   Scientific & Technical Instruments             113.34   
11   KLAC  Technology  Semiconductor Equipment & Materials             111.55   

    1 Month Ago  3 Months Ago  6 Months Ago  Percentile   1 Month Ago  \
19       200.51        169.38        183.58          99            99   
26       145.54        127.62        120.79          96            96   
5        141.14        128.68        143.55          93            89   
17       125.43        111.67        132.49          89            72   
6        141.68        108.64        147.72        

### Misc Test

In [8]:
from vistock import tw

tw.similar_stocks('印度')

[('富邦印度', '00652'), ('富邦印度正2', '00653L'), ('富邦印度反1', '00654R')]

In [9]:
from vistock import tw

tw.get_esb_tickers()[:5]

['9957.TWO', '2646.TWO', '5859.TWO', '6434.TWO', '1480.TWO']